In [32]:
import requests
import json
import fastf1
from matplotlib import pyplot as plt
import fastf1.plotting
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
fastf1.plotting.setup_mpl()
import xgboost as xgb
from get_data import get_lap_data
from sklearn import preprocessing
fastf1.Cache.enable_cache('/Users/marcsperzel/Desktop/F1')  
from constants import DRIVER_NUMBER_TO_DRIVER, LAP_DATA_CATEGORIES, TIME_DATA

In [33]:
PLAYERS_URL = "https://fantasy-api.formula1.com/f1/2022/players"
r = requests.get(PLAYERS_URL)._content
data = json.loads(r.decode("UTF-8"))
players = data['players']
drivers = [player for player in players if player['position']=='Driver']
constructors = [player for player in players if player['position']=='Constructor']
driver_df = pd.DataFrame(drivers).set_index("id")
driver_df['driver_number']=driver_df['last_name'].map(dict((v,k) for k,v in driver_number_to_driver.items()))
constructor_df = pd.DataFrame(constructors).set_index("id")
driver_data_df = pd.DataFrame(driver_df.driver_data.to_list(), index=driver_df.index)
constructor_data_df = pd.DataFrame(constructor_df.constructor_data.to_list(), index=constructor_df.index)

In [82]:
session = fastf1.get_session(2022, 'Bahrain', "Q")
session.load()

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v2.2.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '11', '44', '77', '20', '14', '63', '10', '31', '47', '4', '23', '24', '22', '27', '3', '18', '6', '1']


Hard=0
Medium=1
Soft=2

True=1
False=0

In [24]:
# not the best model
model = xgb.XGBRegressor()
model.load_model('f1_model.json')
lap_data = get_lap_data(session)
predictions = dict(zip(lap_data.index,model.predict(lap_data)))
predictions = {k: v for k, v in sorted(predictions.items(), key=lambda item: item[1],reverse=True)}
{ DRIVER_NUMBER_TO_DRIVER[k]:predictions[k] for k in predictions }